Testing SuperK SELECT control via Python. We are using NKT Photonics SuperK Compact laser with SELECT tunable filter. Laser is on COM3.

On Rayleigh, SDK documents can be found under Users/Public/Public Documents/NKT Photonics

SuperK Compact info is on page 59 of the user manual.

Python interface doesn't work while Control software is connected.

Notes for Compact registers: 

- The module type number is 74h. The standard module address is 1 (01h).
- Emission on/off – the value 0 turns emission off, and 1 turns it on (if interlock circuit
has been reset). The format is an 8-bit unsigned integer 
- Trig mode set with 31h
- Register 0x30 is emission. 1 for on, 0 for off. 
- Interlock register is 32h. A value greater than 1 will reset the interlock circuit if the key is in the "On" position. Sending 0 will disable the interlock. 
- Internal pulse frequency (repetition rate) is 33h. The value is in Hz. Minimum value is 1, maximum is system dependent
- Internal pulse frequency limit readout with 36h. 
- Power level 3Eh. Setpoint value in %. Values input here are automatically converted to repetition rates which overwrites the value in 33h register. Integer value between 0 and 100
- Status bits at 66h
- Readout display text with 78h. 


Register Notes for SuperK SELECT: (page 56 of)
 - module is 67h. Should be at address 16
 - crystal 1 covers VIS/NIR wavelengths, crystal 2 covers NIR/IR

In [3]:
from .NKTP_DLL import *
from .controls import *

ImportError: attempted relative import with no known parent package

In [2]:
import os
os.getcwd()

'c:\\Users\\admin1\\OneDrive\\Documents\\GitHub\\NKT_laser_control'

In [6]:
controls_dir = os.path.dirname(os.path.abspath(controls.__file__))
print("controls.py is located in:", controls_dir)

NameError: name 'controls' is not defined

In [8]:
print(dir(controls))

NameError: name 'controls' is not defined

In [124]:
COMport = 'COM3'
COMPACT_devID = 1
SELECT_devID = 16

mode_mapping = {
    0: 'Internal frequency generator',
    1: 'External trig',
    2: 'Software trigged burst',
    3: 'Hardware trigged burst',
    4: 'External gate on',
    5: 'External gate off',
}

I think the interlock key has to be turned and reset first. Find a way to check interlock status?

In [53]:
emission_on()

Setting emission ON. 0:RegResultSuccess


In [125]:
def trig_mode(mode=None):
    if mode is None:
        result = registerReadU32(COMport, COMPACT_devID, 0x31, -1)
        status = mode_mapping[result[1]]
        print('Laser mode:', status)
    else:
        result = registerWriteReadU32(COMport, COMPACT_devID, 0x31, mode, -1)
        status = mode_mapping[result[1]]
        print('Laser mode:', status)

In [128]:
trig_mode(0)

Laser mode: Internal frequency generator


In [63]:
emission_off()

Setting emission OFF. 0:RegResultSuccess


In [ ]:
# Monitor 1 readout, not working?
result = registerReadU16('COM3', 16, 0x10, -1)
print(result)
print(RegisterResultTypes(result))

(0, 4095)
Unknown result


In [ ]:
# Monitor 2 readout, not working?
result = registerReadU16('COM3', 17, 0x11, -1)
print(result)
print(RegisterResultTypes(result))

(0, 350)
Unknown result


In [70]:
# crystal 1 minimum wavelength, value in pm
result = registerReadU32('COM3', 16, 0x90, -1)
print(result)
print(RegisterResultTypes(result))

(0, 500000)
Unknown result


In [ ]:
def interlock_status():
    if value==0:
        print('Interlock off')
    if value

In [ ]:
status_mapping = {
    0: "Device is off.",
    1: "Device is on.",
    2: "Device is broken."
}

# Function to check the status based on the mapping
def check_device_status(value):
    # Use the get method to provide a default message for unknown values
    message = status_mapping.get(value, "Unknown status.")
    print(message)

# Example usage
status_value = int(input("Enter the device status (0: off, 1: on, 2: broken): "))
check_device_status(status_value)

In [ ]:
interlock_mapping = {
    0: "Interlock off (interlock circuit open).",
    1: "Waiting for interlock reset.",
    3: "Interlock is OK."
}

def interlock_status(value):
    message = interlock_mapping.get(value, "Unknown interlock status.")
    print(message)

 0 Interlock off (interlock circuit open)
0 1 Waiting for interlock reset
0 2 Interlock is OK
1 0 Front panel interlock / key switch off
2 0 Door switch open
3 0 External module interlock
4 0 Interlock power failure
255 - Interlock circuit failure

"All wavelength and amplitude settings are sent to the RF Driver, not to the SuperK SELECT"
First channel is controlled with 0x90, second with 0x91, and so on up to 0x97 for the eighth channel. 
Only use the element on index 0. Can send only one 32 bit integer to change this value. 

In [ ]:
COMport = 'COM3' # depends on the port the device is connected to
COMPACT_devID = 1 # fixed for the SuperK COMPACT
SELECT_devID = 16 # fixed for the SuperK SELECT

def crystal_temp():
    """Read the current crystal temperature in degrees Celsius."""
    result = registerReadS16(COMport, SELECT_devID, 0x38, -1)
    temperature = result[1] # temperature given in 0.1 degrees C
    print('Crystal temperature: ', temperature/10, 'degrees C.', RegisterResultTypes(result))

In [ ]:
def set_wavelength(channel, wavelength):
    """Specify the channel (1-8) and set the wavelength in nm for that channel."""
    channel = int(channel-1)
    address = f"0x9{channel}"
    result = registerWriteReadU32(COMport, SELECT_devID, address, wavelength*1000, -1)
    print('Wavelength set to: ', wavelength/1000, 'nm.', RegisterResultTypes(result))